# DMML : Assignment 2
### Paarth Iyer - MCS202218
### Varun Agrawal - MDS202251

## Task 1

In [33]:
import pandas as pd

Importing the packages used

In [34]:
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score

# Movie dataset - Task 1

Creating bank_y by comparing the revenue and budget for each movie

Creating bolly_X by dropping the Movie name as it will be unique for just about each movie.

As we want to predict budget or flop before the movie is released, Revenue is also dropped.

In [35]:
bolly_data = pd.read_csv(r"./bollywood-data/bollywood-movies.csv")
# bolly_y as our target variable
bolly_y = bolly_data['Revenue(INR)']
bolly_X = bolly_data.drop(columns = ['Movie Name', 'Revenue(INR)'])

#### Preprocessing X
All the 'yes'/'no' fields are label encoded with 'yes'=1 and 'no'=0. 

The 'Release Period' field is encoded with 'Normal' = 0 and 'Holiday' = 1. 

Rest of the columns are then OneHot Encoded

In [36]:
#Encoding Using label encoder
yn_rank = ['no','yes']
yn_encoder = LabelEncoder().fit(yn_rank)
for field in ['Whether Remake', 'Whether Franchise','New Actor', 'New Director', 'New Music Director'] : 
    bolly_X[field] = yn_encoder.fit_transform(bolly_X[field])

bolly_X['Release Period'] = LabelEncoder().fit(['Normal','Holiday']).fit_transform(bolly_X['Release Period'])

bolly_X = pd.get_dummies(bolly_X) # getting dummies for remaining categorical columns

For the OneHot columns, we will drop the ones where the Lead star, Director or Music Director which occur less frequently 

(Counting the lead starts, Directors and Music Directors in the dataset using the Counter object.)

In [37]:
# counting number of times each actor, director, music director appeared in the columns
from collections import Counter
actors = Counter(bolly_data['Lead Star'])
direc = Counter(bolly_data['Director'])
mdirec = Counter(bolly_data['Music Director'])

In [38]:
print(actors.most_common(10))
print(direc.most_common(10))
print(mdirec.most_common(10))

[('Akshay Kumar', 48), ('Ajay Devgn', 35), ('Emraan Hashmi', 27), ('Salman Khan', 27), ('Amitabh Bachchan', 21), ('Saif Ali Khan', 18), ('Sanjay Dutt', 18), ('Shahid Kapoor', 18), ('John Abraham', 17), ('Ranbir Kapoor', 17)]
[('Ram Gopal Verma', 18), ('Vikram Bhatt', 16), ('Priyadarshan', 16), ('Suresh Jain', 15), ('Mohit Suri', 11), ('Rohit Shetty', 11), ('Anurag Kashyap', 10), ('Madhur Bhandarkar', 9), ('Anees Bazmee', 9), ('David Dhawan', 8)]
[('Pritam', 92), ('Himesh Reshammiya', 51), ('Sajid - Wajid', 48), ('Anu Malik', 42), ('Shankar - Ehsaan - Loy', 41), ('Vishal - Shekhar', 38), ('Salim - Sulaiman', 35), ('A.R. Rehman', 26), ('Amit Trivedi', 24), ('Sachin - Jigar', 24)]


In [39]:
#Dropping columns with less frequent values for actor(<4), director(<2), music director(<10)
for act in actors.keys():
    if actors[act] <4:
        bolly_X = bolly_X.drop('Lead Star_'+act, axis=1)

for act in direc.keys():
    if direc[act] <2:
        bolly_X = bolly_X.drop('Director_'+act, axis=1)

for act in mdirec.keys():
    if mdirec[act] <10:
        bolly_X = bolly_X.drop('Music Director_'+act, axis=1)

len(bolly_X.columns)

432

Creating training and test set

In [40]:
#Splitting data with test dataset size 20%
bolly_X_train, bolly_X_test, bolly_y_train, bolly_y_test = sklearn.model_selection.train_test_split(bolly_X, bolly_y, test_size=0.2, random_state=42)

### Decision Tree regressor
Grid search to find the best params

In [41]:
# Generally the  best params are found in these values
param_grids = [{"random_state":[42],"min_samples_leaf":[8,9,10,11], "max_depth":[7,8,9,10], 
                "criterion":["poisson"], "max_features":[0.83, 0.85, 0.87, 0.9]}]

# Using r2 score for grid search
srch = sklearn.model_selection.GridSearchCV(DecisionTreeRegressor(), param_grids, scoring = 'r2', n_jobs=-1)
srch_result = srch.fit(bolly_X_train, bolly_y_train)

Building regressor with those params

In [42]:
#fitting tree with estimator providing best r2 score
tree_bolly = srch_result.best_estimator_
tree_bolly.fit(bolly_X_train, bolly_y_train)

DecisionTreeRegressor(criterion='poisson', max_depth=8, max_features=0.85,
                      min_samples_leaf=10, random_state=42)

The r2 score will provide a good metric to check how much of the variance of the true output is captured by our model

In [43]:
print("R2 score of the model is :",r2_score(bolly_y_test,tree_bolly.predict(bolly_X_test)))

R2 score of the model is : 0.8385394192734265


-----------------------------------------

## Task 2 - bollywood dataset

In [ ]:
import pandas as pd

Importing the packages used

In [ ]:
import sklearn
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

Creating bank_y by comparing the revenue and budget for each movie

Creating bolly_X by dropping the Movie name as it will be unique for just about each movie.

As we want to predict budget or flop before the movie is released, Revenue is also dropped.

In [ ]:
bolly_data = pd.read_csv(r"./bollywood-data/bollywood-movies.csv")
# bolly_y as our target variable
bolly_y = ['hit' if i else 'flop' for i in bolly_data['Revenue(INR)'] > bolly_data['Budget(INR)']]
bolly_X = bolly_data.drop(columns = ['Movie Name', 'Revenue(INR)'])

#### Preprocessing
All the 'yes'/'no' fields are label encoded with 'yes'=1 and 'no'=0. 

The 'Release Period' field is encoded with 'Normal' = 0 and 'Holiday' = 1. 

Rest of the columns are then OneHot Encoded

In [ ]:
yn_rank = ['no','yes']
#Encoding Using label encoder
yn_encoder = LabelEncoder().fit(yn_rank)
for field in ['Whether Remake', 'Whether Franchise','New Actor', 'New Director', 'New Music Director'] : 
    bolly_X[field] = yn_encoder.fit_transform(bolly_X[field])

bolly_X['Release Period'] = LabelEncoder().fit(['Normal','Holiday']).fit_transform(bolly_X['Release Period'])

bolly_X = pd.get_dummies(bolly_X) # getting dummies for remaining categorical columns

For the OneHot columns, we will drop the ones where the Lead star, Director or Music Director which occur less frequently

(Counting the lead starts, Directors and Music Directors in the dataset using the Counter object.)

In [ ]:
# counting number of times each actor, director, music director appeared in the columns
from collections import Counter
actors = Counter(bolly_data['Lead Star'])
direc = Counter(bolly_data['Director'])
mdirec = Counter(bolly_data['Music Director'])

In [ ]:
print(actors.most_common(10))
print(direc.most_common(10))
print(mdirec.most_common(10))

[('Akshay Kumar', 48), ('Ajay Devgn', 35), ('Emraan Hashmi', 27), ('Salman Khan', 27), ('Amitabh Bachchan', 21), ('Saif Ali Khan', 18), ('Sanjay Dutt', 18), ('Shahid Kapoor', 18), ('John Abraham', 17), ('Ranbir Kapoor', 17)]
[('Ram Gopal Verma', 18), ('Vikram Bhatt', 16), ('Priyadarshan', 16), ('Suresh Jain', 15), ('Mohit Suri', 11), ('Rohit Shetty', 11), ('Anurag Kashyap', 10), ('Madhur Bhandarkar', 9), ('Anees Bazmee', 9), ('David Dhawan', 8)]
[('Pritam', 92), ('Himesh Reshammiya', 51), ('Sajid - Wajid', 48), ('Anu Malik', 42), ('Shankar - Ehsaan - Loy', 41), ('Vishal - Shekhar', 38), ('Salim - Sulaiman', 35), ('A.R. Rehman', 26), ('Amit Trivedi', 24), ('Sachin - Jigar', 24)]


In [ ]:
#Dropping columns with less frequent values for actor(<4), director(<2), music director(<8)
for act in actors.keys():
    if actors[act] <4:
        bolly_X = bolly_X.drop('Lead Star_'+act, axis=1)

for act in direc.keys():
    if direc[act] <2:
        bolly_X = bolly_X.drop('Director_'+act, axis=1)

for act in mdirec.keys():
    if mdirec[act] <8:
        bolly_X = bolly_X.drop('Music Director_'+act, axis=1)

len(bolly_X.columns)

447

In [ ]:
#Splitting data with test dataset size 20%
bolly_X_train, bolly_X_test, bolly_y_train, bolly_y_test = sklearn.model_selection.train_test_split(bolly_X, bolly_y, test_size=0.2, random_state=100)

### Decision Tree classifier
Grid search to find the best params

In [ ]:
# Generally the  best params are found in these values
param_grids = [{"random_state":[100],"min_samples_leaf":[8,9,10], "max_depth":[5,6,7,8], 
                "criterion":['gini', 'entropy'], "max_features":[0.87, 0.9, 0.93, 0.96]}]
# Using accuracy for grid search
srch = sklearn.model_selection.GridSearchCV(DecisionTreeClassifier(), param_grids, scoring = "accuracy", n_jobs=-1)
srch_result = srch.fit(bolly_X_train, bolly_y_train)

In [ ]:
#fitting tree with estimator providing best accuracy score
tree_bolly_DTC = srch_result.best_estimator_
tree_bolly_DTC.fit(bolly_X_train,bolly_y_train)

DecisionTreeClassifier(max_depth=6, max_features=0.93, min_samples_leaf=10,
                       random_state=100)

In [ ]:
print(sklearn.metrics.classification_report(bolly_y_test,tree_bolly_DTC.predict(bolly_X_test)))

              precision    recall  f1-score   support

        flop       0.80      0.85      0.83       103
         hit       0.93      0.91      0.92       237

    accuracy                           0.89       340
   macro avg       0.87      0.88      0.87       340
weighted avg       0.89      0.89      0.89       340



## Random Forest classifier

In [ ]:
param_grids = [{"random_state":[100],"n_estimators":[60, 80,100,130],
                "criterion":['gini', 'entropy']}]
# Using accuracy score for grid search
srch = sklearn.model_selection.GridSearchCV(RandomForestClassifier(), param_grids, scoring = "accuracy", n_jobs=-1)
srch_result = srch.fit(bolly_X_train, bolly_y_train)

In [ ]:
srch_result.best_estimator_

RandomForestClassifier(n_estimators=80, random_state=100)

In [ ]:
#fitting rf with estimator providing best accuracy score
tree_bolly_RFC = srch_result.best_estimator_
tree_bolly_RFC.fit(bolly_X_train, bolly_y_train)
print(sklearn.metrics.classification_report(bolly_y_test,tree_bolly_RFC.predict(bolly_X_test)))

              precision    recall  f1-score   support

        flop       0.82      0.80      0.81       103
         hit       0.91      0.92      0.92       237

    accuracy                           0.89       340
   macro avg       0.87      0.86      0.86       340
weighted avg       0.88      0.89      0.88       340



## Boosted decision stumps

In [ ]:
#Using GridSearchCV for getting best hyperparameters
param_grids = [{"random_state":[100],"n_estimators":[8, 10, 13, 15], "learning_rate":[1.4, 1.5, 1.6, 1.7, 1.8], "estimator":[DecisionTreeClassifier(max_depth=1)]}] # decision stump
srch = sklearn.model_selection.GridSearchCV(AdaBoostClassifier(), param_grids, scoring = "accuracy", n_jobs=-1)
srch_result = srch.fit(bolly_X_train, bolly_y_train)

In [ ]:
srch_result.best_estimator_

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=1.6, n_estimators=13, random_state=100)

In [ ]:
#fitting adaboost model with decision stumps with best parameters giving highest accuracy score
tree_bolly_DS = srch_result.best_estimator_
tree_bolly_DS.fit(bolly_X_train, bolly_y_train)
print(sklearn.metrics.classification_report(bolly_y_test,tree_bolly_DS.predict(bolly_X_test)))

              precision    recall  f1-score   support

        flop       0.81      0.84      0.82       103
         hit       0.93      0.91      0.92       237

    accuracy                           0.89       340
   macro avg       0.87      0.88      0.87       340
weighted avg       0.89      0.89      0.89       340



------------------------------

## Task 2 - bank dataset

In [ ]:
import pandas as pd

Importing the packages used

In [ ]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

bank_data loads the data from a ssv to a dataframe using pandas.

While making bank_X, bank_data is copied while dropping the columns 'y', 'duration' and 'day_of_week'

In [ ]:
bank_data = pd.read_csv(r"./bank-data/bank-additional-full.csv",sep=';')
bank_X = bank_data.drop(columns = ['y', 'duration','day_of_week'])

bank_y = [1 if i=='yes' else 0 for i in bank_data.y.values]

THe columns 'housing' and 'loan' are label encoded such that 'yes' = 2, 'unknown' = 1 and 'no'= 0

In [ ]:
# default has too much unknown
# hosuing and loan unknowns are 1to1
yn_rank = ['no','unknown','yes']
yn_encoder = LabelEncoder().fit(yn_rank)
for field in ['housing','loan'] : 
    bank_X[field] = yn_encoder.fit_transform(bank_X[field])

The 'education' column is label encoded using the order below.

After that, rest of the categorical features are OneHot encoded

In [ ]:
# Label encode education with accordance to their level
edu_rank = ["unknown","illiterate","basic.4y","basic.6y","basic.9y","high.school","professional.course","university.degree"]
bank_X["education"] = LabelEncoder().fit(edu_rank).fit_transform(bank_X.education)

bank_X = pd.get_dummies(bank_X) # onehot encoding rest of the categorical features

Seperating training and test data

In [ ]:
bank_X_train, bank_X_test, bank_y_train, bank_y_test = sklearn.model_selection.train_test_split(bank_X, bank_y, random_state=100)

### Decision Tree

In [ ]:
# performing gridsearch on range of params around certain good params found earlier
param_grids = [{"random_state":[42],"min_samples_leaf":[1,6,7, 8], "max_depth":[5,6,7,8], 
                "criterion":['gini', 'entropy'], "max_features":[0.85, 0.87, 0.9, 0.93, 0.96], "class_weight":["balanced"]}]

# using f1 score of 'yes' as a metric to maximize. n_jobs=-1 to make it multi-threaded
srch = sklearn.model_selection.GridSearchCV(DecisionTreeClassifier(), param_grids, scoring = "f1", n_jobs=-1)

srch_result = srch.fit(bank_X_train, bank_y_train)

In [ ]:
# setting the decision tree to the best estimator found
tree_bank_DTC = srch_result.best_estimator_
tree_bank_DTC.fit(bank_X_train, bank_y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=8, max_features=0.93, min_samples_leaf=6,
                       random_state=42)

In [ ]:
print(metrics.classification_report(bank_y_test,tree_bank_DTC.predict(bank_X_test)))

              precision    recall  f1-score   support

           0       0.95      0.84      0.89      9149
           1       0.34      0.64      0.44      1148

    accuracy                           0.82     10297
   macro avg       0.64      0.74      0.67     10297
weighted avg       0.88      0.82      0.84     10297



## Random Forest classifier

In [ ]:
param_grids = [{"random_state":[42],"n_estimators":[100, 120, 130],"max_depth":[12,13,15,None],
                "criterion":['gini', 'entropy']}]

srch = sklearn.model_selection.GridSearchCV(RandomForestClassifier(class_weight="balanced"), param_grids, scoring = "f1", n_jobs=-1)
srch_result = srch.fit(bank_X_train, bank_y_train)

In [ ]:
srch_result.best_estimator_

RandomForestClassifier(class_weight='balanced', max_depth=13, n_estimators=120,
                       random_state=42)

In [ ]:
tree_bank_RFC = srch_result.best_estimator_
tree_bank_RFC.fit(bank_X_train, bank_y_train)
print(metrics.classification_report(bank_y_test,tree_bank_RFC.predict(bank_X_test)))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      9149
           1       0.42      0.57      0.49      1148

    accuracy                           0.87     10297
   macro avg       0.68      0.74      0.70     10297
weighted avg       0.89      0.87      0.87     10297



## Boosted decision stumps

In [ ]:
param_grids = [{"random_state":[42],"n_estimators":[10, 15, 20, 25, 30], "learning_rate":[1.6, 1.8, 1.9, 2.0]}]

srch = sklearn.model_selection.GridSearchCV(AdaBoostClassifier(DecisionTreeClassifier(max_depth=1, class_weight="balanced")), param_grids, scoring = "f1", n_jobs=-1)
srch_result = srch.fit(bank_X_train, bank_y_train)

In [ ]:
srch_result.best_params_

{'learning_rate': 1.9, 'n_estimators': 20, 'random_state': 42}

In [ ]:
tree_bank_DS = srch_result.best_estimator_
tree_bank_DS.fit(bank_X_train, bank_y_train)
print(metrics.classification_report(bank_y_test,tree_bank_DS.predict(bank_X_test)))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      9149
           1       0.41      0.53      0.46      1148

    accuracy                           0.86     10297
   macro avg       0.67      0.72      0.69     10297
weighted avg       0.88      0.86      0.87     10297

